In [ ]:
text = "山路を登りながら、こう考えた。智に働けば角が立つ。情に棹させば流される。意地を通せば窮屈だ。とかくに人の世は住みにくい。"

In [ ]:
#テキスト分類

from transformers import pipeline

classifier = pipeline("text-classification")

In [ ]:
import pandas as pd

outputs = classifier(text) # dict のリスト
pd.DataFrame(outputs)

In [ ]:
#固有表現認識

from transformers import BertForTokenClassification, BertTokenizer

## 日本語 BERT モデル
model_name = "cl-tohoku/bert-base-japanese"
ja_model = BertForTokenClassification.from_pretrained(model_name)
ja_tokenizer = BertTokenizer.from_pretrained(model_name)

## NER 用パイプライン
ner_pipeline = pipeline("ner", model=ja_model, tokenizer=ja_tokenizer)

## ラベルは微妙だが、トークンごとに判定されてそうなことが分かる
ner_results = ner_pipeline(text)
pd.DataFrame(ner_results)

In [ ]:
#質疑応答

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

## 日本語 QA モデル( https://huggingface.co/Mizuiro-sakura/deberta-v2-base-japanese-finetuned-QA より)
tokenizer = AutoTokenizer.from_pretrained('ku-nlp/deberta-v2-base-japanese')
model=AutoModelForQuestionAnswering.from_pretrained('Mizuiro-sakura/deberta-v2-base-japanese-finetuned-QAe')

question = "意地を通すと？"

## 窮屈が抜き出せない
input_ids=tokenizer.encode(question,text) # tokenizerで形態素解析しつつコードに変換する
output= model(torch.tensor([input_ids])) # 学習済みモデルを用いて解析
prediction = tokenizer.decode(input_ids[torch.argmax(output.start_logits): torch.argmax(output.end_logits)]) # 答えに該当する部分を抜き取る
print(prediction)

## 全文が出力されてしまう
# from transformers import BertForQuestionAnswering
# ja_qamodel = BertForQuestionAnswering.from_pretrained(model_name)
# reader = pipeline(task="question-answering", model=ja_qamodel, tokenizer=ja_tokenizer)
# outputs = reader(question=question, context=text)
# pd.DataFrame([outputs])

In [ ]:
#要約

## hugging face で良さげな日本語モデルが見つけられなかった
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, min_length=10, do_sample=False)
print(outputs[0]['summary_text'])

In [ ]:
#翻訳

## 日本語 → 英語
translator = pipeline("translation_ja_to_en", model="Helsinki-NLP/opus-mt-ja-en")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=10)
print(outputs[0]['translation_text'])

In [ ]:
#生成

## 日本語生成モデル( https://huggingface.co/ku-nlp/gpt2-small-japanese-char より)
generator = pipeline("text-generation", model='ku-nlp/gpt2-small-japanese-char')
response = "そういって"
prompt = text + "\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])